In [70]:
# coding: UTF-8
# use model "chinese_roberta_wwm_ext_pytorch"
import torch
import time 
import torch.nn as nn
import torch.nn.functional as F 
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertConfig, BertAdam
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from torch.utils.data import *

path = "data/"
bert_path = "chinese_roberta_wwm_ext_pytorch/"
tokenizer = BertTokenizer(vocab_file=bert_path + "vocab.txt")  # 初始化分词器

In [71]:
input_ids = []     # input char ids
input_types = []   # segment ids
input_masks = []   # attention mask
label = []         # 标签
pad_size = 20     # 也称为 max_len (前期统计分析，文本长度最大值为38，取32即可覆盖99%)
 
with open(path + "train.txt", encoding='utf-8') as f:
    for i, l in tqdm(enumerate(f)): 
        x1, y = l.strip().split('\t')
        x1 = tokenizer.tokenize(x1)
        tokens = ["[CLS]"] + x1 + ["[SEP]"]
        
        # 得到input_id, seg_id, att_mask
        ids = tokenizer.convert_tokens_to_ids(tokens)
        types = [0] *(len(ids))
        masks = [1] * len(ids)
        # 短则补齐，长则切断
        if len(ids) < pad_size:
            types = types + [1] * (pad_size - len(ids))  # mask部分 segment置为1
            masks = masks + [0] * (pad_size - len(ids))
            ids = ids + [0] * (pad_size - len(ids))
        else:
            types = types[:pad_size]
            masks = masks[:pad_size]
            ids = ids[:pad_size]
        input_ids.append(ids)
        input_types.append(types)
        input_masks.append(masks)
#         print(len(ids), len(masks), len(types)) 
        assert len(ids) == len(masks) == len(types) == pad_size
        label.append([int(y)])

8679it [00:00, 12809.71it/s]


In [72]:
# 随机打乱索引
random_order = list(range(len(input_ids)))
np.random.seed(2020)   # 固定种子
np.random.shuffle(random_order)
print(random_order[:10])

# 4:1 划分训练集和测试集
input_ids_train = np.array([input_ids[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_types_train = np.array([input_types[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_masks_train = np.array([input_masks[i] for i in random_order[:int(len(input_ids)*0.8)]])
y_train = np.array([label[i] for i in random_order[:int(len(input_ids) * 0.8)]])
print(input_ids_train.shape, input_types_train.shape, input_masks_train.shape, y_train.shape)

input_ids_test = np.array([input_ids[i] for i in random_order[int(len(input_ids)*0.8):]])
input_types_test = np.array([input_types[i] for i in random_order[int(len(input_ids)*0.8):]])
input_masks_test = np.array([input_masks[i] for i in random_order[int(len(input_ids)*0.8):]])
y_test = np.array([label[i] for i in random_order[int(len(input_ids) * 0.8):]])
print(input_ids_test.shape, input_types_test.shape, input_masks_test.shape, y_test.shape)

[2420, 7581, 7880, 6782, 1290, 3345, 3788, 1280, 7217, 5058]
(6943, 20) (6943, 20) (6943, 20) (6943, 1)
(1736, 20) (1736, 20) (1736, 20) (1736, 1)


In [73]:
BATCH_SIZE = 8
train_data = TensorDataset(torch.LongTensor(input_ids_train), 
                           torch.LongTensor(input_types_train), 
                           torch.LongTensor(input_masks_train), 
                           torch.LongTensor(y_train))
train_sampler = RandomSampler(train_data)  
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(torch.LongTensor(input_ids_test), 
                          torch.LongTensor(input_types_test), 
                         torch.LongTensor(input_masks_test),
                          torch.LongTensor(y_test))
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [74]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(bert_path)  # /bert_pretrain/
        for param in self.bert.parameters():
            param.requires_grad = False  # 每个参数都要 求梯度 feiyu 改成false
        self.fc = nn.Linear(768, 10)   # 768 -> 2

    def forward(self, x):
        context = x[0]  # 输入的句子   (ids, seq_len, mask)
        types = x[1]
        mask = x[2]  # 对padding部分进行mask，和句子相同size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        _, pooled = self.bert(context, token_type_ids=types, 
                              attention_mask=mask, 
                              output_all_encoded_layers=False) # 控制是否输出所有encoder层的结果
        out = self.fc(pooled)   # 得到10分类
        return out

In [75]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'
print(DEVICE)
model = Model().to(DEVICE)

print(model) 

cuda
Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
        

In [76]:
param_optimizer = list(model.named_parameters())  # 模型参数名字列表
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
# optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
NUM_EPOCHS = 3
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-4,
                     warmup=0.05,
                     t_total=len(train_loader) * NUM_EPOCHS
                    )

In [77]:
def train(model, device, train_loader, optimizer, epoch):   # 训练模型
    model.train()
    best_acc = 0.0 
    for batch_idx, (x1,x2,x3, y) in enumerate(train_loader):
        start_time = time.time()
        x1,x2,x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        y_pred = model([x1, x2, x3])  # 得到预测结果
        model.zero_grad()             # 梯度清零
        loss = F.cross_entropy(y_pred, y.squeeze())  # 得到loss
        loss.backward()
        optimizer.step()
        if(batch_idx + 1) % 100 == 0:    # 打印loss
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'.format(epoch, (batch_idx+1) * len(x1), 
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader), 
                                                                           loss.item()))  # 记得为loss.item()

def test(model, device, test_loader):    # 测试模型, 得到测试集评估结果
    model.eval()
    test_loss = 0.0 
    acc = 0 
    for batch_idx, (x1,x2,x3, y) in enumerate(test_loader):
        x1,x2,x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        with torch.no_grad():
            y_ = model([x1,x2,x3])
        test_loss += F.cross_entropy(y_, y.squeeze())
        pred = y_.max(-1, keepdim=True)[1]   # .max(): 2输出，分别为最大值和最大值的index
        acc += pred.eq(y.view_as(pred)).sum().item()    # 记得加item()
    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
          test_loss, acc, len(test_loader.dataset),
          100. * acc / len(test_loader.dataset)))
    return acc / len(test_loader.dataset)

In [78]:
best_acc = 0.0 
PATH = 'roberta_model.pth'  # 定义模型保存路径
for epoch in range(1, NUM_EPOCHS+1):  # 3个epoch
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader)
    if best_acc < acc: 
        best_acc = acc 
        torch.save(model.state_dict(), PATH)  # 保存最优模型
    print("acc is: {:.4f}, best acc is {:.4f}\n".format(acc, best_acc))  

Train Epoch: 1 [800/6943 (11.41%)]	Loss: 0.778597
Train Epoch: 1 [1600/6943 (22.93%)]	Loss: 0.621922
Train Epoch: 1 [2400/6943 (34.45%)]	Loss: 1.100434
Train Epoch: 1 [3200/6943 (45.97%)]	Loss: 0.930870
Train Epoch: 1 [4000/6943 (57.49%)]	Loss: 0.684451
Train Epoch: 1 [4800/6943 (69.01%)]	Loss: 0.737182
Train Epoch: 1 [5600/6943 (80.53%)]	Loss: 0.920716
Train Epoch: 1 [6400/6943 (92.05%)]	Loss: 0.747857

Test set: Average loss: 0.7864, Accuracy: 1194/1736 (68.78%)
acc is: 0.6878, best acc is 0.6878

Train Epoch: 2 [800/6943 (11.41%)]	Loss: 0.716462
Train Epoch: 2 [1600/6943 (22.93%)]	Loss: 0.749733
Train Epoch: 2 [2400/6943 (34.45%)]	Loss: 0.680745
Train Epoch: 2 [3200/6943 (45.97%)]	Loss: 0.700463
Train Epoch: 2 [4000/6943 (57.49%)]	Loss: 0.662157
Train Epoch: 2 [4800/6943 (69.01%)]	Loss: 0.979742
Train Epoch: 2 [5600/6943 (80.53%)]	Loss: 0.561021
Train Epoch: 2 [6400/6943 (92.05%)]	Loss: 0.793773

Test set: Average loss: 0.7488, Accuracy: 1219/1736 (70.22%)
acc is: 0.7022, best acc i

In [80]:
model.load_state_dict(torch.load("roberta_model.pth"))
acc = test(model, DEVICE, test_loader)
if best_acc < acc: 
    best_acc = acc 
    torch.save(model.state_dict(), PATH)  # 保存最优模型
print("acc is: {:.4f}, best acc is {:.4f}\n".format(acc, best_acc))  


# 测试集提交
'''
PATH = "roberta_model.pth"
model.load_state_dict(torch.load(PATH))
def test_for_submit(model, device, test_loader):    # 测试模型
    model.eval()
    preds = []
    for batch_idx, (x1,x2,x3) in tqdm(enumerate(test_loader)):
        x1,x2,x3 = x1.to(device), x2.to(device), x3.to(device)
        with torch.no_grad():
            y_ = model([x1,x2,x3])
        pred = y_.max(-1, keepdim=True)[1].squeeze().cpu().tolist()   
        # .max() 2输出，分别为最大值和最大值的index
        preds.extend(pred) 
    return preds 
preds = test_for_submit(model, DEVICE, test_loader)
'''


Test set: Average loss: 0.7416, Accuracy: 1226/1736 (70.62%)
acc is: 0.7062, best acc is 0.7062



'\nPATH = "roberta_model.pth"\nmodel.load_state_dict(torch.load(PATH))\ndef test_for_submit(model, device, test_loader):    # 测试模型\n    model.eval()\n    preds = []\n    for batch_idx, (x1,x2,x3) in tqdm(enumerate(test_loader)):\n        x1,x2,x3 = x1.to(device), x2.to(device), x3.to(device)\n        with torch.no_grad():\n            y_ = model([x1,x2,x3])\n        pred = y_.max(-1, keepdim=True)[1].squeeze().cpu().tolist()   \n        # .max() 2输出，分别为最大值和最大值的index\n        preds.extend(pred) \n    return preds \npreds = test_for_submit(model, DEVICE, test_loader)\n'